# Assessing robustness of Ward-level measurements

As part of the Mumbai analysis, we are measuring the distance from households to ward centroids. This script is aimed at understanding the accuracy and consistency of centroid measurements compared to the whole AOI

In [1]:
import sys, os, inspect, logging, importlib

import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
from shapely.ops import split, unary_union
from shapely.geometry import box, Point

#Get reference to OSMNX_POIs
sys.path.append(r'../..')
from Market_Access import OSMNX_POIs

#Get reference to GOSTNets
sys.path.append(r'../../GOSTNets/GOSTNets')
import GOSTnet as gn

networkx version: 2.3 
osmnx version: 0.10 


In [14]:
outputFolder = r'C:\Work\Mumbai_OD\GOST_Data'
inputFolder2 = r'C:\Work\Mumbai_OD'
inputFolder = r'./input_folder'

inJobs = os.path.join(inputFolder, "GREATER_MUMBAI.shp")
inHouseholds = os.path.join(inputFolder2, "Home and Work Locations.csv")
inGraph = os.path.join(outputFolder, "OSM_Combo.pickle")

inJ = gpd.read_file(inJobs)
inH = pd.read_csv(inHouseholds)
inH['geometry'] = [Point(x) for x in zip(inH['gps_longitude_home'], inH['gps_latitude_home'])]
inH = gpd.GeoDataFrame(inH, geometry="geometry", crs={'init': 'epsg:4326'})
g = nx.read_gpickle(inGraph)

In [46]:
gn.example_node(g,3)

(0, {'x': 72.8360633305759, 'y': 18.940975234799453, 'geom': <shapely.geometry.point.Point object at 0x0000026A569F8828>})
(1, {'x': 72.83822328429102, 'y': 18.953239981677317, 'geom': <shapely.geometry.point.Point object at 0x0000026A5B5C6828>})
(2, {'x': 72.83947921893387, 'y': 18.960440822411584, 'geom': <shapely.geometry.point.Point object at 0x0000026A5B5C6898>})


In [49]:
type(g)

networkx.classes.multidigraph.MultiDiGraph

In [48]:
type(inH)

geopandas.geodataframe.GeoDataFrame

In [54]:
inH

,household_id,individual_id,female,House No/ Flat No / Building Name,street,ward_home,sample point area,gps_reading_home,gps_latitude_home,gps_longitude_home,...,gps_accuracy_home,gps_satellites_home,city,pin_home,Do you work?-q9_1,Ward in Which Person Works,PIN code in which person works,geometry,x,y
0,65002,65002_1,0,25,NEW LINKING ROAD,16,650,Yes,19.206719,72.834494,...,7.0,9.0,MUMBAI,400067,"Yes, outside home",16.0,400067,POINT (72.83449399999999 19.206719),72.834494,19.206719
1,65002,65002_2,1,25,NEW LINKING ROAD,16,650,Yes,19.206719,72.834494,...,7.0,9.0,MUMBAI,400067,No,NaN,NaN,POINT (72.83449399999999 19.206719),72.834494,19.206719
2,65002,65002_3,1,25,NEW LINKING ROAD,16,650,Yes,19.206719,72.834494,...,7.0,9.0,MUMBAI,400067,NaN,NaN,NaN,POINT (72.83449399999999 19.206719),72.834494,19.206719
3,65002,65002_4,1,25,NEW LINKING ROAD,16,650,Yes,19.206719,72.834494,...,7.0,9.0,MUMBAI,400067,NaN,NaN,NaN,POINT (72.83449399999999 19.206719),72.834494,19.206719
4,19102,19102_1,0,66,JSW ROAD,17,191,Yes,19.072440,72.861138,...,8.0,7.0,MUMBAI,400098,"Yes, outside home",17.0,400098,POINT (72.861138 19.07244),72.861138,19.072440
5,19102,19102_2,1,66,JSW ROAD,17,191,Yes,19.072440,72.861138,...,8.0,7.0,MUMBAI,400098,No,NaN,NaN,POINT (72.861138 19.07244),72.861138,19.072440
6,19102,19102_4,1,66,JSW ROAD,17,191,Yes,19.072440,72.861138,...,8.0,7.0,MUMBAI,400098,NaN,NaN,NaN,POINT (72.861138 19.07244),72.861138,19.072440
7,19102,19102_3,1,66,JSW ROAD,17,191,Yes,19.072440,72.861138,...,8.0,7.0,MUMBAI,400098,NaN,NaN,NaN,POINT (72.861138 19.07244),72.861138,19.072440
8,52404,52404_2,1,267,AZAD HIND SOCIETY,19,524,Yes,19.180492,72.871748,...,7.0,11.0,MUMBAI,400097,No,NaN,NaN,POINT (72.871748 19.180492),72.871748,19.180492
9,52404,52404_3,0,267,AZAD HIND SOCIETY,19,524,Yes,19.180492,72.871748,...,7.0,11.0,MUMBAI,400097,NaN,NaN,NaN,POINT (72.871748 19.180492),72.871748,19.180492


In [62]:
sampleWard = inJ.iloc[0]
# for the sample ward, identify the network node locations and measure travel time to them
snappedHH = gn.pandana_snap(g, inH, add_dist_to_node_col=False)

IndexError: positional indexers are out-of-bounds

In [50]:
node_gdf = gn.node_gdf_from_graph(g)
in_df = inH
from scipy import spatial

In [70]:
node_gdf

,node_ID,osmid,highway,x,geom,y,geometry
0,0,NaN,NaN,72.836063,POINT (72.8360633305759 18.94097523479945),18.940975,POINT (72.8360633305759 18.94097523479945)
1,1,NaN,NaN,72.838223,POINT (72.83822328429102 18.95323998167732),18.953240,POINT (72.83822328429102 18.95323998167732)
2,2,NaN,NaN,72.839479,POINT (72.83947921893387 18.96044082241158),18.960441,POINT (72.83947921893387 18.96044082241158)
3,3,NaN,NaN,72.843532,POINT (72.84353211300539 19.01751641049149),19.017516,POINT (72.84353211300539 19.01751641049149)
4,4,NaN,NaN,72.837661,POINT (72.8376614 19.0094817),19.009482,POINT (72.8376614 19.0094817)
5,5,NaN,NaN,72.844344,POINT (72.84434387789514 18.96732091413225),18.967321,POINT (72.84434387789514 18.96732091413225)
6,6,NaN,NaN,72.844144,POINT (72.84414412090767 18.97812083602538),18.978121,POINT (72.84414412090767 18.97812083602538)
7,7,NaN,NaN,72.844087,POINT (72.8440871427147 18.987273139187),18.987273,POINT (72.8440871427147 18.987273139187)
8,8,NaN,NaN,72.854422,POINT (72.8544218 18.99877959999999),18.998780,POINT (72.8544218 18.99877959999999)
9,9,NaN,NaN,72.832905,POINT (72.83290483671482 18.9952211592503),18.995221,POINT (72.83290483671482 18.9952211592503)


In [64]:
in_df['x'] = in_df.geometry.x
in_df['y'] = in_df.geometry.y
G_tree = spatial.KDTree(node_gdf[['x','y']].as_matrix())

C:\WBG\Anaconda\envs\gost\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [77]:
distances, indices = G_tree.query(node_gdf[['x','y']].as_matrix())

C:\WBG\Anaconda\envs\gost\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [80]:
max(indices)

26686

In [74]:
max(indices)

26687

In [55]:
in_df

,household_id,individual_id,female,House No/ Flat No / Building Name,street,ward_home,sample point area,gps_reading_home,gps_latitude_home,gps_longitude_home,...,gps_accuracy_home,gps_satellites_home,city,pin_home,Do you work?-q9_1,Ward in Which Person Works,PIN code in which person works,geometry,x,y
0,65002,65002_1,0,25,NEW LINKING ROAD,16,650,Yes,19.206719,72.834494,...,7.0,9.0,MUMBAI,400067,"Yes, outside home",16.0,400067,POINT (72.83449399999999 19.206719),72.834494,19.206719
1,65002,65002_2,1,25,NEW LINKING ROAD,16,650,Yes,19.206719,72.834494,...,7.0,9.0,MUMBAI,400067,No,NaN,NaN,POINT (72.83449399999999 19.206719),72.834494,19.206719
2,65002,65002_3,1,25,NEW LINKING ROAD,16,650,Yes,19.206719,72.834494,...,7.0,9.0,MUMBAI,400067,NaN,NaN,NaN,POINT (72.83449399999999 19.206719),72.834494,19.206719
3,65002,65002_4,1,25,NEW LINKING ROAD,16,650,Yes,19.206719,72.834494,...,7.0,9.0,MUMBAI,400067,NaN,NaN,NaN,POINT (72.83449399999999 19.206719),72.834494,19.206719
4,19102,19102_1,0,66,JSW ROAD,17,191,Yes,19.072440,72.861138,...,8.0,7.0,MUMBAI,400098,"Yes, outside home",17.0,400098,POINT (72.861138 19.07244),72.861138,19.072440
5,19102,19102_2,1,66,JSW ROAD,17,191,Yes,19.072440,72.861138,...,8.0,7.0,MUMBAI,400098,No,NaN,NaN,POINT (72.861138 19.07244),72.861138,19.072440
6,19102,19102_4,1,66,JSW ROAD,17,191,Yes,19.072440,72.861138,...,8.0,7.0,MUMBAI,400098,NaN,NaN,NaN,POINT (72.861138 19.07244),72.861138,19.072440
7,19102,19102_3,1,66,JSW ROAD,17,191,Yes,19.072440,72.861138,...,8.0,7.0,MUMBAI,400098,NaN,NaN,NaN,POINT (72.861138 19.07244),72.861138,19.072440
8,52404,52404_2,1,267,AZAD HIND SOCIETY,19,524,Yes,19.180492,72.871748,...,7.0,11.0,MUMBAI,400097,No,NaN,NaN,POINT (72.871748 19.180492),72.871748,19.180492
9,52404,52404_3,0,267,AZAD HIND SOCIETY,19,524,Yes,19.180492,72.871748,...,7.0,11.0,MUMBAI,400097,NaN,NaN,NaN,POINT (72.871748 19.180492),72.871748,19.180492


In [73]:
node_gdf['node_ID'].iloc[26686]
#node_gdf['node_ID'].iloc[26687]

26686

In [56]:
#in_df['NN'] = list(node_gdf['node_ID'].iloc[indices])

C:\WBG\Anaconda\envs\gost\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [65]:
in_df['NN'] = indices

In [66]:
in_df

,household_id,individual_id,female,House No/ Flat No / Building Name,street,ward_home,sample point area,gps_reading_home,gps_latitude_home,gps_longitude_home,...,gps_satellites_home,city,pin_home,Do you work?-q9_1,Ward in Which Person Works,PIN code in which person works,geometry,x,y,NN
0,65002,65002_1,0,25,NEW LINKING ROAD,16,650,Yes,19.206719,72.834494,...,9.0,MUMBAI,400067,"Yes, outside home",16.0,400067,POINT (72.83449399999999 19.206719),72.834494,19.206719,3706
1,65002,65002_2,1,25,NEW LINKING ROAD,16,650,Yes,19.206719,72.834494,...,9.0,MUMBAI,400067,No,NaN,NaN,POINT (72.83449399999999 19.206719),72.834494,19.206719,3706
2,65002,65002_3,1,25,NEW LINKING ROAD,16,650,Yes,19.206719,72.834494,...,9.0,MUMBAI,400067,NaN,NaN,NaN,POINT (72.83449399999999 19.206719),72.834494,19.206719,3706
3,65002,65002_4,1,25,NEW LINKING ROAD,16,650,Yes,19.206719,72.834494,...,9.0,MUMBAI,400067,NaN,NaN,NaN,POINT (72.83449399999999 19.206719),72.834494,19.206719,3706
4,19102,19102_1,0,66,JSW ROAD,17,191,Yes,19.072440,72.861138,...,7.0,MUMBAI,400098,"Yes, outside home",17.0,400098,POINT (72.861138 19.07244),72.861138,19.072440,11462
5,19102,19102_2,1,66,JSW ROAD,17,191,Yes,19.072440,72.861138,...,7.0,MUMBAI,400098,No,NaN,NaN,POINT (72.861138 19.07244),72.861138,19.072440,11462
6,19102,19102_4,1,66,JSW ROAD,17,191,Yes,19.072440,72.861138,...,7.0,MUMBAI,400098,NaN,NaN,NaN,POINT (72.861138 19.07244),72.861138,19.072440,11462
7,19102,19102_3,1,66,JSW ROAD,17,191,Yes,19.072440,72.861138,...,7.0,MUMBAI,400098,NaN,NaN,NaN,POINT (72.861138 19.07244),72.861138,19.072440,11462
8,52404,52404_2,1,267,AZAD HIND SOCIETY,19,524,Yes,19.180492,72.871748,...,11.0,MUMBAI,400097,No,NaN,NaN,POINT (72.871748 19.180492),72.871748,19.180492,21690
9,52404,52404_3,0,267,AZAD HIND SOCIETY,19,524,Yes,19.180492,72.871748,...,11.0,MUMBAI,400097,NaN,NaN,NaN,POINT (72.871748 19.180492),72.871748,19.180492,21690
